In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec  8 01:21:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import glob
import math
warnings.filterwarnings('ignore')

In [44]:
train = pd.read_csv('drive/MyDrive/dacon/data_re/상추/train_log_1203.csv').iloc[:,1:]
test = pd.read_csv('drive/MyDrive/dacon/data_re/상추/test_log_1203.csv').iloc[:,1:]

In [8]:
train.columns

Index(['내부온도관측치0~2', '내부온도관측치1~3', '내부온도관측치2~4', '내부온도관측치3~5', '내부온도관측치4~6',
       '내부온도관측치5~7', '내부온도관측치6~8', '내부온도관측치7~9', '내부온도관측치8~10', '내부온도관측치9~11',
       ...
       'day_red_weight_std', 'day_blue_weight_std', 'day_light_weight_std',
       'day_water_weight_mean', 'day_white_weight_mean', 'day_red_weight_mean',
       'day_blue_weight_mean', 'day_light_weight_mean', 'DAT',
       'predicted_weight_g'],
      dtype='object', length=250)

In [13]:
dropc = []
col = train.columns
for i in col :
  if '적색' in i :
    dropc.append(i)
  elif '청색' in i :
    dropc.append(i)
  elif '내부온도관측치' in i:
    dropc.append(i)
  elif '시간당백색' in i :
    dropc.append(i)

In [ ]:
!pip install lightgbm==3.3.2
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [14]:
import lightgbm as lgb

In [15]:
lgb.__version__

'3.3.2'

In [47]:
drop_clumns = train.drop(dropc,axis=1)

In [48]:
drop_clumns_t = test.drop(dropc,axis=1)

In [49]:
X = drop_clumns.drop(['predicted_weight_g'],axis=1)
y = drop_clumns['predicted_weight_g']

In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

In [60]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
params = {'learning_rate': 0.033, 
              'boosting': 'gbdt', 
              'objective': 'regression', 
              'metric': 'rmse', 
              'is_training_metric': True, 
              'num_leaves': 50, 
              'feature_fraction': 0.9, 
              'bagging_fraction': 0.7, 
              'bagging_freq': 5, 
              'seed':404}
X = drop_clumns.drop(['predicted_weight_g'],axis=1)
y = drop_clumns['predicted_weight_g']
model= {}
for f in range(0,28):
      train_idx = [i for i in range(0,784)]
      valid_idx = train_idx[f*28:f*28+28]
      for idx in valid_idx:
        train_idx.remove(idx)
      print(f'===================================={f+1}============================================')
      print(min(valid_idx),max(valid_idx))
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[valid_idx], y.iloc[train_idx], y.iloc[valid_idx]
      
      train_ds = lgb.Dataset(x_train, label=y_train) 
      valid_ds = lgb.Dataset(x_val, label=y_val) 

      m = lgb.train(params, train_ds,1000, valid_ds, verbose_eval=0, early_stopping_rounds=50) 
      
      y_pred = m.predict(x_val)
      rmae = mean_squared_error(y_val, y_pred)**0.5
      print(f"{f + 1} Fold rMAE = {rmae}")
      model[f] = m
      print(f'================================================================================\n\n')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [61]:
testX = drop_clumns_t.drop(['predicted_weight_g'],axis=1)
pc=[0]*140
for fold in range(28):
    pc += model[fold].predict(testX)/28
pc

array([ 6.67869603,  6.88287931,  7.26141296,  7.3641347 ,  7.93764402,
        8.83245803,  9.38950385,  9.72552375, 10.95069114, 12.04135882,
       14.16570898, 15.53867235, 34.92594404, 22.50740613, 17.76749982,
       29.22872644, 30.01623146, 21.4104905 , 23.83612645, 33.9580385 ,
       43.18230405, 52.55651086, 62.40735328, 56.17116572, 62.51374888,
       73.07028104, 73.72214563, 81.87373243,  6.60936011,  6.8875753 ,
        7.19146346,  7.11526938,  7.5379475 ,  8.19339093,  8.47969434,
        9.37272181,  9.42813215, 10.56032596, 11.53957117, 13.97064616,
       15.33926193, 16.87574347, 21.21996461, 23.22059449, 25.73111436,
       34.75948015, 34.82149513, 40.66379361, 42.93102129, 50.91849465,
       64.98292498, 61.82675663, 75.84589034, 61.90274212, 67.57012658,
       68.15352054,  6.93371203,  6.85526864,  6.9556253 ,  7.13343146,
        7.30134279,  7.74481742,  8.120377  ,  8.74601001,  9.30342458,
        9.78201185, 10.2841979 , 10.38398589, 11.26354888, 11.78

In [55]:
all_target_list = sorted(glob.glob('drive/MyDrive/dacon/data_re/상추/test_target/*.csv'))
i=0
for test_path in all_target_list:
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = pc[i*28:i*28+28]
    submit_df.to_csv(test_path, index=False)
    i+=1